In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [14]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [16]:
df = df.drop(columns=['kwid' , 'meta'])
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75107 entries, 0 to 75113
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Question  75107 non-null  object 
 1   Answer    75107 non-null  object 
 2   dirt      75107 non-null  float64
 3   humor     75107 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.9+ MB


In [17]:
df.sample(5)

,Question,Answer,dirt,humor
14650,How can you tell if she is virgin or not?,Paddy was planning to get married and asked hi...,0.00,0.2
9224,Why does Michael Scott hate Hamlet?,Toby or not Toby,0.00,0.2
28818,Why don't Mexicans cross the border in groups?,"Because the border says ""No TRESpassing""",0.60,0.2
47266,Why did two owls get arrested for the same crime?,They were in ca*hoots*.,0.00,0.2
44038,Why does everyone hate Hitler?,The man was a hero... After all he did kill Hi...,0.25,0.2
